In [1]:
import pybullet as p
import pybullet_data
import time
import math
import numpy as np
from pinocchio_template import thneed

# Connect to the physics server
physicsClient = p.connect(p.GUI)
p.resetDebugVisualizerCamera(cameraDistance=2.0, cameraYaw=0, cameraPitch=-30, cameraTargetPosition=[0,0,0])

pybullet build time: Jan 29 2025 23:16:28


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3080/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 535.183.01
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 535.183.01
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3080/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


In [ ]:
friction_thing_N = 0
gravity = True
position_control = False
disable_velocity_control = not position_control

# Set up the environment
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.81*gravity)
planeId = p.loadURDF("/home/a2rlab/Documents/bullet3/data/plane.urdf")

# robotId = p.loadURDF("/home/a2rlab/Documents/bullet3/examples/pybullet/gym/pybullet_data/franka_panda/panda.urdf", [0, 0, 0], useFixedBase=1)
robotId = p.loadURDF("/home/a2rlab/Documents/emre/indy-ros2/indy_description/urdf_files/indy7_nolimits.urdf", [0, 0, 0], useFixedBase=1, flags=p.URDF_USE_INERTIA_FROM_FILE)

num_joints = p.getNumJoints(robotId)
print(f"Number of joints: {num_joints}")

# Print joint info
for i in range(num_joints):
    joint_info = p.getJointInfo(robotId, i)
    print(f"Joint {i}, Name: {joint_info[1].decode('utf-8')}, Type: {joint_info[2]}")

if disable_velocity_control:
    p.setJointMotorControlArray(bodyIndex=robotId,
                            jointIndices=[i for i in range(num_joints)],
                            controlMode=p.VELOCITY_CONTROL,
                            forces=friction_thing_N*np.ones(num_joints))

sim_timestep = 0.01
solve_timestep = 0.01
p.setTimeStep(sim_timestep) # Default is 1/240 seconds

t = thneed(dt=solve_timestep)
if not gravity:
    t.gravity_off()

t.max_qp_iters = 5

xs = 0.1 * np.hstack([np.ones(6), np.zeros(6)])
for i in range(1,7):
    p.resetJointState(robotId, i, xs[i-1], 0)
# goal = np.array([-2.30313706e-01, -1.23076760e-01,  6.17299293e-02])
# goal = np.array([ 1.51468905e-10, -1.86500000e-01,  1.26750000e+00]) # p sure this is 0
goal = t.eepos(1 * np.ones(6))
start = t.eepos(xs[:t.nq])

goal_trace = []
tracelen = 330
for i in range(tracelen):
    # goal_trace.append((1/tracelen)*((tracelen - i)*start + (i)*goal)) # linear interpolation
    goal_trace.append(goal)
goal_trace = np.array(goal_trace).reshape(-1)

all_xs = []
all_ctrl = []

running_fails = 0
successes = 0
for i in range(300):
    all_xs.append(np.array(xs))
    print(np.linalg.norm(t.eepos(xs[:6]) - goal_trace[3*(i+t.N-1):3*(i+t.N)]))
    

    # if np.linalg.norm(t.eepos(xs[:6]) - goal) < 5e-2:
    #     break

    # t.clean_start()
    # t.shift_start()
    
    t.XU[:t.nx] = xs  # set start state, this helps with finding good min

    if t.sqp(xs, goal_trace[3*i:3*(i+t.N)]):
        print("FAIL")
        running_fails +=1
        if running_fails==t.N-1:
            raise Exception(f"ran out of traj, successes: {successes}")
    else:
        successes +=1
        running_fails = 0
    
    if position_control and 0:
        o = 1
        p.setJointMotorControlArray(bodyIndex=robotId,
                                    jointIndices=[i for i in range(1,7)],
                                    controlMode=p.POSITION_CONTROL,
                                    targetPositions=t.XU[o*t.nxu:o*t.nxu+t.nq], # + 0.3 * np.random.rand(6),
                                    # targetVelocities=t.XU[o*t.nxu+t.nq:o*t.nxu+t.nx],
                                    forces=[1e5 for _ in range(6)]
                                    )
    else:
        o = 0
        trqs = t.XU[o*t.nxu+t.nx:o*t.nxu+t.nxu]
        all_ctrl.append(trqs)
        p.setJointMotorControlArray(bodyIndex=robotId,
                                jointIndices=[i for i in range(1,7)],
                                controlMode=p.TORQUE_CONTROL,
                                forces=trqs)
    
    
    # Step the simulation forward
    for i in range(int(solve_timestep / sim_timestep)):
        p.stepSimulation()
        time.sleep(sim_timestep)
    

    joint_states = p.getJointStates(robotId, [i for i in range(1,7)])
    q = [j[0] for j in joint_states]
    v = [j[1] for j in joint_states]
    torques_applied = [j[3] for j in joint_states] # this is zeroed for torque control


    xs[:t.nq] = q
    xs[t.nq:t.nx] = v


# Disconnect from the physics server
# p.disconnect()

ven = NVIDIA Corporation
Number of joints: 8
Joint 0, Name: global, Type: 4
Joint 1, Name: joint0, Type: 0
Joint 2, Name: joint1, Type: 0
Joint 3, Name: joint2, Type: 0
Joint 4, Name: joint3, Type: 0
Joint 5, Name: joint4, Type: 0
Joint 6, Name: joint5, Type: 0
Joint 7, Name: tcp, Type: 4
ven = NVIDIA Corporation
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frame
b3Printf: b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:

b3Printf: tcp
osqp warm starting: True
1.0285452357455207
0.02008342742919922
1.0256781060502813
0.0204007625579834
1.0202520210779789
0.02865123748779297
1.0121486735573968
0.027187347412109375
1.0014635995653969
0.020208120346069336
0.9899120892347195
0.02238631248474121
0.9777701327553412
0.02619171142578125
0.9648218778235477
0.02263164520263672
0.9508407210118341
FAIL
0.0293428897857666
0.935817306

Exception: ran out of traj, successes: 58

In [ ]:
all_xs = np.array(all_xs)
all_ctrl = np.array(all_ctrl)

In [ ]:
print(np.max(all_xs, axis=0) - np.min(all_xs, axis=0))
print(np.max(all_ctrl, axis=0) - np.min(all_ctrl, axis=0))

[0.92112651 0.6914019  0.82602766 0.34671275 0.4610423  0.39161852 5.20675803 1.69933365 3.57613378 1.86808912 5.08177854 4.19535459]
[9.37625764e+01 2.52787761e+02 1.41607899e+02 2.69569546e+01 3.57804210e+01 1.87452774e-01]


In [ ]:
print(sum(t.stats['linesearch_alphas']['values']) / len(t.stats['linesearch_alphas']['values']))
len([a for a in t.stats['linesearch_alphas']['values'] if a==0.0])

0.1376358695652174


160

In [ ]:
all_ctrl

array([[ 6.48383057e+01,  1.39867331e+02,  9.59777480e+01, -1.47000452e+01,  2.20405734e+01,  7.45929908e-02],
       [ 4.84291020e+01,  1.52299241e+02,  8.16191140e+01, -1.10209912e+01,  2.43554455e+01,  1.35011537e-01],
       [ 4.58076212e+01,  1.40489148e+02,  7.41168851e+01, -9.40130914e+00,  2.29185208e+01,  1.35804754e-01],
       [ 4.01168378e+01,  1.25627471e+02,  6.35247807e+01, -7.29668183e+00,  2.13187164e+01,  1.42877820e-01],
       [-2.89242706e+01,  9.51202930e+01,  2.67738850e+01, -8.47199017e+00,  2.05964535e+01,  1.85316026e-01],
       [-1.68966063e+01,  6.55339333e+01,  1.68852086e+01, -3.21356793e+00,  1.73104405e+01,  1.85547681e-01],
       [-1.09144681e+01,  6.53712590e+01,  2.56225238e+01, -4.21040875e+00,  1.75457861e+01,  1.68684277e-01],
       [-2.28822080e-01,  4.47674663e+01,  2.69237783e+01, -3.65607971e+00,  1.57879805e+01,  1.40484752e-01],
       [-2.28822080e-01,  4.47674663e+01,  2.69237783e+01, -3.65607971e+00,  1.57879805e+01,  1.40484752e-01],
 